In [1]:
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
import string
import jsonrpclib
from simplejson import loads
server = jsonrpclib.Server("http://localhost:8080")
replace_punctuation = string.maketrans(string.punctuation, ' '*len(string.punctuation))

In [2]:
annotation = open('./annotation_speaker.new')

text = annotation.readline()
print text

annotation.close()

    This was the closest Davos had come to His Grace in the eight days since Black Betha had joined the rest of the fleet off Storm's End. He'd sought an audience within an hour of his arrival, only to be told that the king was occupied. The king was often occupied, Davos learned from his son Devan, one of the royal squires. Now that Stannis Baratheon had come into his power, the lordlings buzzed around him like flies round a corpse. He looks half a corpse too, years older than when I left Dragonstone. Devan said the king scarcely slept of late. "Since Lord Renly died, he has been troubled by terrible nightmares," the boy had confided to his father. "Maester's potions do not touch them. Only the Lady Melisandre can soothe him to sleep."



In [3]:
f = open('./speechverbs')

speechverbs = []
for line in f:
    if line != '\n':
        speechverbs.append(line.split('\n')[0].lower())
print speechverbs

['address', 'admit', 'advise', 'agree', 'analyse', 'announce', 'answer', 'appeal', 'argue', 'articulate', 'ask', 'assure', 'babble', 'bark', 'bawl', 'beg', 'bellow', 'bemoan', 'blabber', 'blather', 'bleat', 'blubber', 'bluff', 'blurt', 'bluster', 'boast', 'brag', 'breathe', 'cackle', 'call', 'chant', 'chatter', 'check', 'cheer', 'chime', 'chirp', 'chirrup', 'chortle', 'chuckle', 'clarify', 'cluck', 'coax', 'comment', 'complain', 'concede', 'confess', 'confide', 'confirm', 'consent', 'convey', 'coo', 'correct', 'couch', 'cough', 'critique', 'crow', 'cry', 'decide', 'declare', 'delineate', 'demand', 'describe', 'disagree', 'drawl', 'drone', 'entreat', 'exclaim', 'expand', 'explain', 'express', 'falter', 'forward', 'fuss', 'giggle', 'goad', 'groan', 'grumble', 'growl', 'grunt', 'hint', 'hiss', 'holler', 'hoot', 'howl', 'hum', 'illustrate', 'implore', 'inform', 'insist', 'interrupt', 'intimate', 'intone', 'jabber', 'jibber', 'judge', 'laud', 'laugh', 'lecture', 'lie', 'maintain', 'mention'

In [4]:
verbs = []
lemmas = []
position_verbs = []
result = loads(server.parse(text))
for i in range(len(result['sentences'])):
    for obj in result['sentences'][i]['parsetree'].split('] ['):
        if 'PartOfSpeech=VB' in obj:
            lemmas.append(obj.split(' ')[4].split('=')[1])
            verbs.append(obj.split(' ')[0].split('=')[1])
            position_verbs.append(obj.split(' ')[1].split('=')[1])
            
for unit in verbs:
    print unit

was
had
come
had
joined
sought
be
told
was
occupied
was
occupied
learned
had
come
buzzed
round
looks
left
said
slept
died
has
been
troubled
had
confided
do
touch
soothe
sleep


In [5]:
for i in range(len(result['sentences'])):
    for unit in result['sentences'][i]['dependencies']:
        print unit

[u'root', u'ROOT', u'closest']
[u'nsubj', u'closest', u'This']
[u'cop', u'closest', u'was']
[u'det', u'closest', u'the']
[u'nsubj', u'come', u'Davos']
[u'aux', u'come', u'had']
[u'acl:relcl', u'closest', u'come']
[u'case', u'Grace', u'to']
[u'nmod:poss', u'Grace', u'His']
[u'nmod:to', u'come', u'Grace']
[u'case', u'days', u'in']
[u'det', u'days', u'the']
[u'nummod', u'days', u'eight']
[u'nmod:in', u'Grace', u'days']
[u'mark', u'joined', u'since']
[u'amod', u'Betha', u'Black']
[u'nsubj', u'joined', u'Betha']
[u'aux', u'joined', u'had']
[u'advcl', u'come', u'joined']
[u'det', u'rest', u'the']
[u'dobj', u'joined', u'rest']
[u'case', u'fleet', u'of']
[u'det', u'fleet', u'the']
[u'nmod:of', u'rest', u'fleet']
[u'case', u'End', u'off']
[u'nmod:poss', u'End', u'Storm']
[u'case', u'Storm', u"'s"]
[u'nmod:off', u'joined', u'End']
[u'root', u'ROOT', u'sought']
[u'nsubj', u'sought', u'He']
[u'aux', u'sought', u"'d"]
[u'det', u'audience', u'an']
[u'dobj', u'sought', u'audience']
[u'case', u'hour',

In [6]:
feature = []
position_quotation = []
position = 0
start = 0
for c in text:
    if c != ' ':
        start = 1
    if c == '"':
        position_quotation.append(position)
    if start == 1:
        position = position+1
distance = []
list_filter = []
n = 0
for obj in position_verbs:
    if len(position_quotation) == 1:
        distance.append(int(obj)-position_quotation[0])
    else:
        stat = 0
        writed = 0
        for i in range(len(position_quotation)):
            if i%2 == 0:
                dist1 = int(obj)-position_quotation[i]
            elif i%2 == 1:
                dist2 = int(obj)-position_quotation[i]
                stat = 1
            if stat == 1:
                if dist1 * dist2 <0:
                    list_filter.append(n)
                if writed == 0:
                    distance.append((abs(dist1)+abs(dist2)-(position_quotation[i]-position_quotation[i-1]-1))/2)
                stat = 0
                writed = 1
    n = n+1

for index in sorted(list_filter, reverse=True):
    del position_verbs[index]
    del distance[index]
    del verbs[index]
    del lemmas[index]

'''print list_filter
print len(position_verbs)
print distance
print len(verbs)
print position_quotation'''


for i in range(len(verbs)):
    feature.append([])
    feature[i].append(len(verbs))
    if lemmas[i].lower() in speechverbs:
        feature[i].append(1)
    elif lemmas[i].lower() not in speechverbs:
        feature[i].append(0)
    feature[i].append(distance[i])
    
print feature

[[23, 0, 543], [23, 0, 521], [23, 0, 517], [23, 0, 463], [23, 0, 459], [23, 0, 408], [23, 0, 350], [23, 1, 347], [23, 0, 328], [23, 0, 324], [23, 0, 305], [23, 0, 295], [23, 0, 279], [23, 0, 198], [23, 0, 194], [23, 0, 159], [23, 0, 130], [23, 0, 111], [23, 0, 62], [23, 1, 38], [23, 0, 15], [23, 0, 10], [23, 1, 14]]


Features:

subject in the utterance or not

verb in the speechverbs list or not

the subject is person or not

if no subject is person try with objects or Possessive Pronouns

the subject is little possible to be multipersons

distance between verbes and utterances